In [ ]:
import datetime as dt

import mplfinance as mpf
import pandas as pd
import numpy as np
import pandera as pa
import matplotlib.pyplot as plt

import shared


logger = shared.get_logger()

In [ ]:
klines_btc_1m_df = shared.read_binance_klines()

In [ ]:
def find_double_tops_from_to_linear(df: pd.DataFrame) -> list[list[tuple[str, int]]]:
    """Figures out dounble top based on prev ATH without digging into every candle.
    
    Examples:
    alines_map = find_double_tops_from_to_linear(df=slice_df)
    """
    rows = list(df.iterrows())
    out = {}
    i = 0
    while i<len(rows):
        j=i
        while j<len(rows) and rows[i][1].high >= rows[j][1].high:
            j+=1
        if j == len(rows):
            break
        key = rows[i][1].open_time
        x = (rows[i][0], rows[i][1].high)
        y = (rows[j][0], rows[i][1].high)
        out[key] = [x, y]
        i = j
    return out

In [ ]:
klines_btc_15m_df = shared.split_df(df=klines_btc_1m_df, interval_str='15m')
klines_btc_1h_df = shared.split_df(df=klines_btc_1m_df, interval_str='1h')

In [ ]:
mpf.plot(klines_btc_15m_df.iloc[0:50], type='candle', figsize=(14, 4), style=shared.s)

In [ ]:
mpf.plot(klines_btc_1h_df, type='candle', warn_too_much_data=len(klines_btc_1h_df), figsize=(200, 200/3.5), style=shared.s)

In [ ]:
class DoubleTopSchema(pa.DataFrameModel):
    date: pa.typing.Index[dt.datetime]
    open_time: pa.typing.Series[float]
    open: pa.typing.Series[float]
    close: pa.typing.Series[float]
    low: pa.typing.Series[float]
    high: pa.typing.Series[float]


@pa.check_types
def find_double_tops_from_to_lth(
    df: pa.typing.DataFrame[DoubleTopSchema],
    gain_threshold: float,
    interval_threshold: str,
) -> list[list[tuple[str, int]]]:
    rows = list(df.iterrows())
    out = {}
    for i in range(len(rows)):
        j=i+1
        while j<len(rows) and rows[i][1].high >= rows[j][1].high:
            j+=1
        if j == len(rows):
            continue
        key = rows[i][1].open_time
        x = (rows[i][0], rows[i][1].close)
        y = (rows[j][0], rows[i][1].close)
        max_gain = (rows[j][1].high - x[1]) / x[1]
        if max_gain < gain_threshold:
            continue
        interval = rows[j][1].open_time - rows[i][1].open_time
        if interval > shared.interval_secs_map[interval_threshold]*1000:
            continue
        out[key] = [x, y]
    return out


slice_df = klines_btc_15m_df
alines_map = find_double_tops_from_to_lth(
    df=slice_df,
    gain_threshold=0.003,
    interval_threshold='1h',
)
fig, _ = mpf.plot(
    slice_df,
    type='candle',
    warn_too_much_data=len(slice_df),
    figsize=(14*15, 4*15),
    alines={'alines': list(alines_map.values()), 'colors': ['#f19d38']},
    style=shared.s,
    returnfig=True,
)